# OpenAI Pydantic Program

Generate structured data with [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) via LlamaIndex.  

In [1]:
from pydantic import BaseModel
from typing import List

from llama_index.program import OpenAIPydanticProgram

Define output schema

In [2]:
class Song(BaseModel):
    title: str
    length_seconds: int
    
class Album(BaseModel):
    name: str
    artist: str
    songs: List[Song]

Define openai pydantic program

In [3]:
program = OpenAIPydanticProgram.from_defaults(
    output_cls=Album, 
    prompt_template_str="Generate an example album, with an artist and a list of songs. Using the movie {movie_name} as inspiration",
    verbose=True,
)

Run program to get structured output.  

In [4]:
output = program(movie_name='The Shining')

Function call: output_pydantic with args: {
  "name": "The Shining",
  "artist": "Wendy Carlos",
  "songs": [
    {
      "title": "Main Title",
      "length_seconds": 180
    },
    {
      "title": "Rocky Mountains",
      "length_seconds": 240
    },
    {
      "title": "Lullaby",
      "length_seconds": 150
    },
    {
      "title": "Music for Strings, Percussion and Celesta",
      "length_seconds": 300
    },
    {
      "title": "Midnight",
      "length_seconds": 210
    }
  ]
}


The output is a valid Pydantic object that we can then use to call functions/APIs. 

In [5]:
output

Album(name='The Shining', artist='Wendy Carlos', songs=[Song(title='Main Title', length_seconds=180), Song(title='Rocky Mountains', length_seconds=240), Song(title='Lullaby', length_seconds=150), Song(title='Music for Strings, Percussion and Celesta', length_seconds=300), Song(title='Midnight', length_seconds=210)])